In [1]:
import json
import pandas as pd
import urllib3
import SimpleITK as sitk
import numpy as np
import csv
import os

In [2]:
# ITKSNAP segementation description file format
output_columns = {
'IDX': 'uint16', # Zero-based index 
'-R-': 'uint8', # Red color component (0..255)
'-G-': 'uint8', # Green color component (0..255)
'-B-': 'uint8', # Blue color component (0..255)
'-A-': 'float', # Label transparency (0.00 .. 1.00)
'VIS': 'uint8', # Label visibility (0 or 1)
'MSH': 'uint8', # Label mesh visibility (0 or 1)
'LABEL': 'str' #Label description
}

In [3]:
# specify the structure graph from Allen API
graph_id = 1

In [4]:
# specify the location of the input annotation directory
input_path = "C:/Users/lydia/OneDrive/Work/DigitalAssets/converted_mouse_ccf/annotation/ccf_2017/"
output_path = "C:/Users/lydia/OneDrive/Work/DigitalAssets/converted_mouse_ccf/annotation/ccf_2017/ITKSnap_Random_Color"

In [5]:
if not os.path.exists(output_path) :
    os.makedirs(output_path)

In [6]:
# RMA query to fetch structures for the structure graph
query_url = "http://api.brain-map.org/api/v2/data/query.json?criteria=model::Structure"
query_url += ",rma::criteria,[graph_id$eq%d]" % graph_id
query_url += ",rma::options[order$eq'structures.id'][num_rows$eqall]"

# Make http request and create a pandas dataframe
http = urllib3.PoolManager()
r = http.request('GET', query_url)
data = json.loads(r.data.decode('utf-8'))['msg']
structures = pd.read_json( json.dumps(data) )


In [7]:
# ITKSNAP description file columns
structures['IDX'] = structures.index.values + 1  # simply add one to index
structures['-A-'] = 1 # make structures opaque
structures['VIS'] = 1 # make all structures visible
structures['MSH'] = 1 # make all meshes visible
structures['LABEL'] = structures['acronym'] + ' - ' + structures['id'].apply(str)

np.random.seed(657)
for c in ['-R-','-G-','-B-'] :
    structures[c] = np.rint(np.random.uniform(0,255,size=len(structures)))
    structures[c] = structures[c].astype(int)

In [8]:
output_index_file = os.path.join(output_path, 'itksnap_label_description.txt')
structures.to_csv( output_index_file, 
                        sep=' ', columns=output_columns, header=False, index=False, float_format='%.0f' )

In [9]:
#
# convert each file in the input directory
#

lut = np.zeros(int(max(structures.id.values)) + 1, np.uint32)
lut[structures.id] = structures.IDX

for f in os.scandir( input_path ):
    if f.path.endswith(".nii.gz") and f.is_file():
        
        print("converting %s" % f.path)
        
        inputImage = sitk.ReadImage(f.path)
        arr = sitk.GetArrayFromImage(inputImage)
        arr = np.take( lut, arr )
        outputImage = sitk.GetImageFromArray( arr )
        outputImage.CopyInformation( inputImage )
        
        output_file = os.path.join( output_path, os.path.basename(f))
        sitk.WriteImage( outputImage, output_file, True )
        


converting C:/Users/lydia/OneDrive/Work/DigitalAssets/converted_mouse_ccf/annotation/ccf_2017/annotation_10.nii.gz
converting C:/Users/lydia/OneDrive/Work/DigitalAssets/converted_mouse_ccf/annotation/ccf_2017/annotation_100.nii.gz
converting C:/Users/lydia/OneDrive/Work/DigitalAssets/converted_mouse_ccf/annotation/ccf_2017/annotation_25.nii.gz
converting C:/Users/lydia/OneDrive/Work/DigitalAssets/converted_mouse_ccf/annotation/ccf_2017/annotation_50.nii.gz
